In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
# Set the start and end dates for the data
start_date = '1950-01-01'
end_date = '2023-12-31'

# S&P 500 and Nasdaq Composite Streak Analysis

In [ ]:
# Download S&P 500 data from Yahoo Finance
sp500_data = yf.download('^GSPC', start=start_date, end=end_date, interval='1wk')

# Download Nasdaq Composite data from Yahoo Finance
nasdaq_data = yf.download('^IXIC', start=start_date, end=end_date, interval='1wk')

# Resample the daily data to weekly frequency and set the index to the end of the week
sp500_data_weekly = sp500_data.resample('W-Fri').last()
nasdaq_data_weekly = nasdaq_data.resample('W-Fri').last()

# Calculate weekly returns
sp500_data_weekly['Weekly_Return'] = sp500_data_weekly['Adj Close'].pct_change()
nasdaq_data_weekly['Weekly_Return'] = nasdaq_data_weekly['Adj Close'].pct_change()

# Create columns for consecutive weekly gains and declines
sp500_data_weekly['Gains'] = (sp500_data_weekly['Weekly_Return'] > 0).astype(int)
sp500_data_weekly['Declines'] = (sp500_data_weekly['Weekly_Return'] < 0).astype(int)
nasdaq_data_weekly['Gains'] = (nasdaq_data_weekly['Weekly_Return'] > 0).astype(int)
nasdaq_data_weekly['Declines'] = (nasdaq_data_weekly['Weekly_Return'] < 0).astype(int)

# Create columns for streaks
sp500_data_weekly['Streak_Gains'] = sp500_data_weekly['Gains'].groupby((sp500_data_weekly['Gains'] != sp500_data_weekly['Gains'].shift(1)).cumsum()).cumsum()
sp500_data_weekly['Streak_Declines'] = sp500_data_weekly['Declines'].groupby((sp500_data_weekly['Declines'] != sp500_data_weekly['Declines'].shift(1)).cumsum()).cumsum()
nasdaq_data_weekly['Streak_Gains'] = nasdaq_data_weekly['Gains'].groupby((nasdaq_data_weekly['Gains'] != nasdaq_data_weekly['Gains'].shift(1)).cumsum()).cumsum()
nasdaq_data_weekly['Streak_Declines'] = nasdaq_data_weekly['Declines'].groupby((nasdaq_data_weekly['Declines'] != nasdaq_data_weekly['Declines'].shift(1)).cumsum()).cumsum()

# Merge the DataFrames
combined_data = pd.merge(sp500_data_weekly[['Adj Close', 'Weekly_Return', 'Streak_Gains', 'Streak_Declines']],
                          nasdaq_data_weekly[['Adj Close', 'Weekly_Return', 'Streak_Gains', 'Streak_Declines']],
                          how='outer',
                          left_index=True,
                          right_index=True,
                          suffixes=('_SP500', '_Nasdaq'))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
combined_data

,Adj Close_SP500,Weekly_Return_SP500,Streak_Gains_SP500,Streak_Declines_SP500,Adj Close_Nasdaq,Weekly_Return_Nasdaq,Streak_Gains_Nasdaq,Streak_Declines_Nasdaq
Date,,,,,,,,
1950-01-06,17.090000,NaN,0,0,NaN,NaN,NaN,NaN
1950-01-13,16.650000,-0.025746,0,1,NaN,NaN,NaN,NaN
1950-01-20,16.940001,0.017417,1,0,NaN,NaN,NaN,NaN
1950-01-27,16.900000,-0.002361,0,1,NaN,NaN,NaN,NaN
1950-02-03,17.350000,0.026627,1,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2023-12-01,4594.629883,0.007740,5,0,14305.030273,0.003802,5.0,0.0
2023-12-08,4604.370117,0.002120,6,0,14403.969727,0.006916,6.0,0.0
2023-12-15,4719.189941,0.024937,7,0,14813.919922,0.028461,7.0,0.0


### Example analysis - weeks where S&P 500 saw 9 straight weekly gains

In [ ]:
combined_data[combined_data["Streak_Gains_SP500"] >= 9]

,Adj Close_SP500,Weekly_Return_SP500,Streak_Gains_SP500,Streak_Declines_SP500,Adj Close_Nasdaq,Weekly_Return_Nasdaq,Streak_Gains_Nasdaq,Streak_Declines_Nasdaq
Date,,,,,,,,
1951-02-09,22.209999,0.005888,9,0,NaN,NaN,NaN,NaN
1957-05-17,47.150002,0.012020,9,0,NaN,NaN,NaN,NaN
1957-05-24,47.209999,0.001272,10,0,NaN,NaN,NaN,NaN
1957-05-31,47.430000,0.004660,11,0,NaN,NaN,NaN,NaN
1957-06-07,47.849998,0.008855,12,0,NaN,NaN,NaN,NaN
1957-06-14,48.150002,0.006270,13,0,NaN,NaN,NaN,NaN
1958-10-17,51.459999,0.001362,9,0,NaN,NaN,NaN,NaN
1961-02-03,62.220001,0.016003,9,0,NaN,NaN,NaN,NaN
1963-05-03,70.029999,0.004735,9,0,NaN,NaN,NaN,NaN
